In [7]:
import pandas as pd

# Cargar datos
archivo_entrada = '../vehicles_us.csv'
archivo_salida = '../vehicles_us_limpio_2.csv'
df = pd.read_csv(archivo_entrada)

# Llenar valores faltantes
df['model'].fillna('Otro', inplace=True)
df['paint_color'].fillna('Otro', inplace=True)
df['model_year'] = df['model_year'].fillna(0).astype(int)
df['odometer'] = df['odometer'].fillna(0).astype(int)
df['cylinders'] = df['cylinders'].fillna(0).astype(int)
df['is_4wd'] = df['is_4wd'].fillna(3).astype(int)

# Mapear 4WD
is_4wd_map = {1: 'Sí', 0: 'Sí', 3: '-'}
df['is_4wd'] = df['is_4wd'].map(is_4wd_map).fillna('No').astype(str)

# Dividir columna model
df[['Marca', 'Modelo']] = (
    df['model']
    .str.extract(r'^(\w+)\s+(.*)', expand=True)
    .apply(lambda col: col.str.strip().str.title())
)
df.drop(columns='model', inplace=True)

# Renombrar columnas
df.rename(columns={
    'price': 'Precio',
    'model_year': 'Año del modelo',
    'condition': 'Condición',
    'cylinders': 'Cilindrada',
    'fuel': 'Combustible',
    'odometer': 'Kilometraje',
    'transmission': 'Transmisión',
    'type': 'Tipo',
    'paint_color': 'Color',
    'is_4wd': '4WD',
    'date_posted': 'Fecha de posteo',
    'days_listed': 'Días publicado'
}, inplace=True)

# Mapeos
cond_map = {
    'new': '*****',
    'like new': '****',
    'excellent': '***',
    'good': '**',
    'fair': '*',
    'salvage': '-'
}
df['Condición'] = df['Condición'].map(cond_map).fillna('-')

fuel_map = {
    'diesel': 'Diesel',
    'electric': 'Eléctrico',
    'gas': 'Gasolina',
    'hybrid': 'Híbrido',
    'other': 'Otro'
}
df['Combustible'] = df['Combustible'].map(fuel_map).fillna('Otro')

trans_map = {
    'automatic': 'Automático',
    'manual': 'Manual',
    'other': 'Otro'
}
df['Transmisión'] = df['Transmisión'].map(trans_map).fillna('Otro')

df['Tipo'] = df['Tipo'].str.title().fillna('Otro')
df['Tipo'] = df['Tipo'].replace({
    'Mini-Van': 'Mini-Van',
    'Suv': 'SUV',
    'Offroad': 'Off-Road',
    'Truck': 'Camioneta',
    'Bus': 'Autobus'
})

color_map = {
    'black': 'Negro',
    'white': 'Blanco',
    'silver': 'Plata',
    'blue': 'Azul',
    'red': 'Rojo',
    'grey': 'Gris',
    'green': 'Verde',
    'brown': 'Marrón',
    'orange': 'Naranja',
    'purple': 'Morado',
    'yellow': 'Amarillo',
    'custom': 'Personalizado',
    'other': 'Otro'
}
df['Color'] = df['Color'].map(color_map).fillna('Otro')
df['Color'] = df['Color'].apply(lambda x: x.capitalize())

df['Fecha de posteo'] = pd.to_datetime(df['Fecha de posteo'], errors='coerce')
df['Días publicado'] = df['Días publicado'].fillna('-')

# --- CORRECCIÓN DE PRECIOS MENORES A 100 ---
conteo_total = 0
conteo_por_condicion = {}
conteo_global = 0

# Separar autos válidos para referencia
df_validos = df[df['Precio'] >= 100]

for idx, fila in df[df['Precio'] < 100].iterrows():
    marca = fila['Marca']
    modelo = fila['Modelo']
    tipo = fila['Tipo']
    
    condiciones = [
        (marca, modelo, tipo),
        (marca, modelo, None),
        (marca, None, tipo),
        (None, modelo, tipo)
    ]

    nueva_mediana = None
    for cond in condiciones:
        subset = df_validos.copy()
        if cond[0]: subset = subset[subset['Marca'] == cond[0]]
        if cond[1]: subset = subset[subset['Modelo'] == cond[1]]
        if cond[2]: subset = subset[subset['Tipo'] == cond[2]]
        if not subset.empty:
            nueva_mediana = subset['Precio'].median()
            clave = f"Marca={cond[0] or '*'}, Modelo={cond[1] or '*'}, Tipo={cond[2] or '*'}"
            conteo_por_condicion[clave] = conteo_por_condicion.get(clave, 0) + 1
            break

    if nueva_mediana is None:
        nueva_mediana = df_validos['Precio'].median()
        conteo_global += 1

    df.at[idx, 'Precio'] = int(nueva_mediana)
    conteo_total += 1

# Reordenar columnas
columnas_ordenadas = [
    'Precio', 'Marca', 'Modelo', 'Año del modelo', 
    'Condición', 'Cilindrada', 'Combustible', 'Kilometraje', 
    'Transmisión', 'Tipo', '4WD', 'Color', 
    'Fecha de posteo', 'Días publicado'
]
df = df[columnas_ordenadas]

# Guardar archivo final
print(f"\n✅ Guardando archivo limpio en: {archivo_salida}")
df.to_csv(archivo_salida, index=False)

# Mostrar resumen
print(f"\n📊 Se corrigieron {conteo_total} precios menores a 100.\n")
print("📌 Distribución por condiciones usadas:")
for clave, valor in sorted(conteo_por_condicion.items(), key=lambda x: -x[1]):
    print(f"  - {clave}: {valor} autos")
print(f"\n🟡 Autos que usaron mediana global: {conteo_global}")


/tmp/ipykernel_80268/871634912.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['model'].fillna('Otro', inplace=True)
/tmp/ipykernel_80268/871634912.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '


✅ Guardando archivo limpio en: ../vehicles_us_limpio_2.csv

📊 Se corrigieron 863 precios menores a 100.

📌 Distribución por condiciones usadas:
  - Marca=Dodge, Modelo=Charger, Tipo=Sedan: 77 autos
  - Marca=Ram, Modelo=3500, Tipo=Camioneta: 76 autos
  - Marca=Ford, Modelo=F-150, Tipo=Camioneta: 60 autos
  - Marca=Jeep, Modelo=Wrangler, Tipo=SUV: 55 autos
  - Marca=Chevrolet, Modelo=Camaro, Tipo=Coupe: 46 autos
  - Marca=Toyota, Modelo=Tundra, Tipo=Camioneta: 46 autos
  - Marca=Chevrolet, Modelo=Tahoe, Tipo=SUV: 37 autos
  - Marca=Nissan, Modelo=Rogue, Tipo=SUV: 36 autos
  - Marca=Jeep, Modelo=Grand Cherokee, Tipo=SUV: 34 autos
  - Marca=Ram, Modelo=1500, Tipo=Camioneta: 34 autos
  - Marca=Ford, Modelo=Mustang, Tipo=Coupe: 18 autos
  - Marca=Gmc, Modelo=Sierra 1500, Tipo=Camioneta: 17 autos
  - Marca=Gmc, Modelo=Acadia, Tipo=SUV: 15 autos
  - Marca=Chevrolet, Modelo=Suburban, Tipo=SUV: 15 autos
  - Marca=Nissan, Modelo=Altima, Tipo=Sedan: 14 autos
  - Marca=Honda, Modelo=Civic, Tipo=S